In [ ]:
import numpy as np
import pandas as pd
import fastai.vision.all as faiv
import torch
import sys

sys.path.append('../input/pytorch-image-models')

In [ ]:
model_1 = torch.load('../input/cv-models-1/SWIN_v2_Fold_1.pkl')
model_2 = torch.load('../input/cv-models-1/SWIN_v2_Fold_2.pkl')
model_3 = torch.load('../input/cv-models-1/SWIN_v2_Fold_3.pkl')
model_4 = torch.load('../input/cv-models-2/SWIN_v2_Fold_4.pkl')
model_5 = torch.load('../input/cv-models-2/SWIN_v2_Fold_5.pkl')
models = [model_1, model_2, model_3, model_4, model_5]

In [ ]:
test_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv') 
test_df['Id'] = test_df['Id'].apply(lambda x: f"../input/petfinder-pawpularity-score/test/{x}.jpg")
test_df = test_df.rename(columns={'Id': 'Path Id'})
test_df['Pawpularity'] = [1 for i in range(len(test_df))]

fastai_dl_test = faiv.ImageDataLoaders.from_df(df = test_df,
                                     seed = 1997,
                                     fn_col = 'Path Id',
                                     label_col = 'Pawpularity',
                                     y_block = faiv.RegressionBlock,
                                     item_tfms= faiv.Resize(224))

In [ ]:
test_dl = fastai_dl_test.test_dl(test_df)
pred_vals = []
for i in range(5):
    
    model = models[i]
    preds, eggs = model.tta(dl = test_dl, n = 1, beta = 1)
    pred_vals += [preds.numpy().T]
    
all_preds = 100*np.mean(np.vstack(pred_vals), axis=0)

In [ ]:
sample_df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
sample_df['Pawpularity'] = all_preds
sample_df.to_csv('submission.csv',index=False)

np.vstack(pred_vals)